In [ ]:
# load the following national variables from the World Bank database using -wbgapi-
#    BAR.SCHL.2024 - average years of education completed among people age 20-24
#    BAR.SCHL.2024.FE - average years of education completed among people age 20-24
#    SP.POP.TOTL - total population
#    for years 1960, 1980, 2000, 2010
#    hint: use -wbgapi- options -skipAggs=True- and -columns='series'-
# calculate female education as a percent of male education, age 20-24
#    note that since females and males are ~50% of the population,
#       BAR.SCHL.2024 = 1/2*BAR.SCHL.2024.FE + 1/2*BAR.SCHL.2024.MA
#    since male education ≡ BAR.SCHL.2024.MA isn't in the database, 
#       BAR.SCHL.2024.MA = 2*BAR.SCHL.2024 - BAR.SCHL.2024.FE
# create a -pivot_table- of average female % of male education for 
#    different income level countries in rows, and
#    4 different time periods in columns
# save as an HTML file

In [23]:
# load the pandas and numpy packages to work with data
import pandas as pd
import numpy as np
# load the "wbgapi" package to download WDI data series
import wbgapi as wb

# ignore depreciation warnings

In [ ]:
# use this if you are not interested about depreciation warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# load data
it is not in a nice format. Maybe there is some options in the method for a different format. However, will see how fix this using pandas instead. 

In [152]:
# examples for -wbgapi- commands at https://pypi.org/project/wbgapi/
# download WDI data series for national CO2 emmissions per person & population into a dataframe
df = wb.data.DataFrame(['BAR.SCHL.2024','BAR.SCHL.2024.FE', 'SP.POP.TOTL'], time=[1960, 1980, 2000, 2010], skipAggs=True, db=12)
# skipAggs=True skips entries for regional aggregates (i.e. only country observations)
# columns='series' would put multiple years in rows rather than in separate columns
df

YR1960      YR1980       YR2000       YR2010
economy series                                                        
ABW     BAR.SCHL.2024        NaN         NaN          NaN          NaN
        BAR.SCHL.2024.FE     NaN         NaN          NaN          NaN
        SP.POP.TOTL          NaN    60096.00     90853.00    101669.00
AFG     BAR.SCHL.2024       0.55        2.22         3.66         4.58
        BAR.SCHL.2024.FE    0.09        0.73         1.80         2.33
...                          ...         ...          ...          ...
ZMB     BAR.SCHL.2024.FE    2.70        5.28         7.11         7.05
        SP.POP.TOTL          NaN  5851825.00  10415944.00  13605984.00
ZWE     BAR.SCHL.2024       3.29        6.34         9.03         8.71
        BAR.SCHL.2024.FE    2.48        5.11         8.85         8.85
        SP.POP.TOTL          NaN  7408624.00  11881477.00  12697723.00

[678 rows x 4 columns]

# moving row values as column names. 
Running below cell shows us that there is a multindex (nested index). 
```
        level=0    level=1 
                   level=-1  <-- -1 shortcut for last index in the multindex
names=['economy', 'series'], length=678)
```
We need to bring up the level=1 index to be as variable names. This is achieved with [`unstack(level=1)`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html#pandas-dataframe-unstack). This brings the values as columns. On contrary [`stack(level=<level>)`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html#pandas-dataframe-stack)
brings columns into row values. 

In [156]:
df.index


MultiIndex([('ABW',    'BAR.SCHL.2024'),
            ('ABW', 'BAR.SCHL.2024.FE'),
            ('ABW',      'SP.POP.TOTL'),
            ('AFG',    'BAR.SCHL.2024'),
            ('AFG', 'BAR.SCHL.2024.FE'),
            ('AFG',      'SP.POP.TOTL'),
            ('AGO',    'BAR.SCHL.2024'),
            ('AGO', 'BAR.SCHL.2024.FE'),
            ('AGO',      'SP.POP.TOTL'),
            ('AIA',    'BAR.SCHL.2024'),
            ...
            ('YEM',      'SP.POP.TOTL'),
            ('ZAF',    'BAR.SCHL.2024'),
            ('ZAF', 'BAR.SCHL.2024.FE'),
            ('ZAF',      'SP.POP.TOTL'),
            ('ZMB',    'BAR.SCHL.2024'),
            ('ZMB', 'BAR.SCHL.2024.FE'),
            ('ZMB',      'SP.POP.TOTL'),
            ('ZWE',    'BAR.SCHL.2024'),
            ('ZWE', 'BAR.SCHL.2024.FE'),
            ('ZWE',      'SP.POP.TOTL')],
           names=['economy', 'series'], length=678)

In [157]:
df.unstack(level=1)

YR1960                                     YR1980  \
series  BAR.SCHL.2024 BAR.SCHL.2024.FE SP.POP.TOTL BAR.SCHL.2024   
economy                                                            
ABW               NaN              NaN         NaN           NaN   
AFG              0.55             0.09         NaN          2.22   
AGO               NaN              NaN         NaN           NaN   
AIA               NaN              NaN         NaN           NaN   
ALB              6.12             5.36         NaN          9.35   
...               ...              ...         ...           ...   
XKX               NaN              NaN         NaN           NaN   
YEM              0.01             0.01         NaN          0.36   
ZAF              4.86             4.97         NaN          5.77   
ZMB              3.34             2.70         NaN          5.92   
ZWE              3.29             2.48         NaN          6.34   

                                            YR2000                   \
series  BAR.SCHL.2024.FE SP.POP.TOTL BAR.SCHL.2024 BAR.SCHL.2024.FE   
economy                                                               
ABW                  NaN     60096.0           NaN              NaN   
AFG                 0.73  13356511.0          3.66             1.80   
AGO                  NaN   8341289.0           NaN              NaN   
AIA                  NaN         NaN           NaN              NaN   
ALB                 9.18   2671997.0         10.61            10.73   
...                  ...         ...           ...              ...   
XKX                  NaN   1521000.0           NaN              NaN   
YEM                 0.13   7941898.0          4.03             2.25   
ZAF                 5.36  28556769.0          9.31             9.42   
ZMB                 5.28   5851825.0          7.46             7.11   
ZWE                 5.11   7408624.0          9.03             8.85   

                           YR2010                               
series  SP.POP.TOTL BAR.SCHL.2024 BAR.SCHL.2024.FE SP.POP.TOTL  
economy                                                         
ABW         90853.0           NaN              NaN    101669.0  
AFG      20779953.0          4.58             2.33  29185507.0  
AGO      16395473.0           NaN              NaN  23356246.0  
AIA             NaN           NaN              NaN         NaN  
ALB       3089027.0         11.12            11.19   2913021.0  
...             ...           ...              ...         ...  
XKX       1700000.0           NaN              NaN   1775680.0  
YEM      17409072.0          5.24             4.55  23154855.0  
ZAF      44967708.0         11.08            11.23  51216964.0  
ZMB      10415944.0          7.35             7.05  13605984.0  
ZWE      11881477.0          8.71             8.85  12697723.0  

[226 rows x 12 columns]

# moving columns into row values


In [169]:

# non multindex implies level=0
display(df.columns)
# If we already did the unstack(level=1) transformation, then we would have a multindex.
# However, Year is still in level=0. 
display(df.unstack(level=1).columns)
# we will use stack() in both cases to get the same result. 
display(df.stack(level=0))
display(df.unstack(level=1).stack(level=0)) # associative property of stack and unstack
display(df.stack(level=0).unstack(level=1))

Index(['YR1960', 'YR1980', 'YR2000', 'YR2010'], dtype='object')

MultiIndex([('YR1960',    'BAR.SCHL.2024'),
            ('YR1960', 'BAR.SCHL.2024.FE'),
            ('YR1960',      'SP.POP.TOTL'),
            ('YR1980',    'BAR.SCHL.2024'),
            ('YR1980', 'BAR.SCHL.2024.FE'),
            ('YR1980',      'SP.POP.TOTL'),
            ('YR2000',    'BAR.SCHL.2024'),
            ('YR2000', 'BAR.SCHL.2024.FE'),
            ('YR2000',      'SP.POP.TOTL'),
            ('YR2010',    'BAR.SCHL.2024'),
            ('YR2010', 'BAR.SCHL.2024.FE'),
            ('YR2010',      'SP.POP.TOTL')],
           names=[None, 'series'])

economy  series                  
ABW      SP.POP.TOTL       YR1980       60096.00
                           YR2000       90853.00
                           YR2010      101669.00
AFG      BAR.SCHL.2024     YR1960           0.55
                           YR1980           2.22
                                        ...     
ZWE      BAR.SCHL.2024.FE  YR2000           8.85
                           YR2010           8.85
         SP.POP.TOTL       YR1980     7408624.00
                           YR2000    11881477.00
                           YR2010    12697723.00
Length: 1802, dtype: float64

series          BAR.SCHL.2024  BAR.SCHL.2024.FE  SP.POP.TOTL
economy                                                     
ABW     YR1980            NaN               NaN      60096.0
        YR2000            NaN               NaN      90853.0
        YR2010            NaN               NaN     101669.0
AFG     YR1960           0.55              0.09          NaN
        YR1980           2.22              0.73   13356511.0
...                       ...               ...          ...
ZMB     YR2010           7.35              7.05   13605984.0
ZWE     YR1960           3.29              2.48          NaN
        YR1980           6.34              5.11    7408624.0
        YR2000           9.03              8.85   11881477.0
        YR2010           8.71              8.85   12697723.0

[795 rows x 3 columns]

series          BAR.SCHL.2024  BAR.SCHL.2024.FE  SP.POP.TOTL
economy                                                     
ABW     YR1980            NaN               NaN      60096.0
        YR2000            NaN               NaN      90853.0
        YR2010            NaN               NaN     101669.0
AFG     YR1960           0.55              0.09          NaN
        YR1980           2.22              0.73   13356511.0
...                       ...               ...          ...
ZMB     YR2010           7.35              7.05   13605984.0
ZWE     YR1960           3.29              2.48          NaN
        YR1980           6.34              5.11    7408624.0
        YR2000           9.03              8.85   11881477.0
        YR2010           8.71              8.85   12697723.0

[795 rows x 3 columns]

# reset all indexes
final step will be to reset the economy index that is left

In [173]:
df.stack(level=0).unstack(level=1).reset_index()

series,economy,level_1,BAR.SCHL.2024,BAR.SCHL.2024.FE,SP.POP.TOTL
0,ABW,YR1980,NaN,NaN,60096.0
1,ABW,YR2000,NaN,NaN,90853.0
2,ABW,YR2010,NaN,NaN,101669.0
3,AFG,YR1960,0.55,0.09,NaN
4,AFG,YR1980,2.22,0.73,13356511.0
...,...,...,...,...,...
790,ZMB,YR2010,7.35,7.05,13605984.0
791,ZWE,YR1960,3.29,2.48,NaN
792,ZWE,YR1980,6.34,5.11,7408624.0
793,ZWE,YR2000,9.03,8.85,11881477.0


In [172]:
# the column axis is labeled as 'series' from the unstack(level=1) transformation.
# we can remove this with the rename_axis method. 
df.stack(level=0).unstack(level=1).reset_index().rename_axis(columns=None)

,economy,level_1,BAR.SCHL.2024,BAR.SCHL.2024.FE,SP.POP.TOTL
0,ABW,YR1980,NaN,NaN,60096.0
1,ABW,YR2000,NaN,NaN,90853.0
2,ABW,YR2010,NaN,NaN,101669.0
3,AFG,YR1960,0.55,0.09,NaN
4,AFG,YR1980,2.22,0.73,13356511.0
...,...,...,...,...,...
790,ZMB,YR2010,7.35,7.05,13605984.0
791,ZWE,YR1960,3.29,2.48,NaN
792,ZWE,YR1980,6.34,5.11,7408624.0
793,ZWE,YR2000,9.03,8.85,11881477.0


# putting it all together

In [149]:

df2 = df.unstack(level=1).stack(level=0, future_stack=True).reset_index().rename_axis(None, axis="columns")
df2['year'] = df2['level_1'].str.extract(r'YR(\d{4})')
df2.drop(columns=['level_1'], inplace=True)
df2


,economy,BAR.SCHL.2024,BAR.SCHL.2024.FE,SP.POP.TOTL,year
0,ABW,NaN,NaN,NaN,1960
1,ABW,NaN,NaN,60096.0,1980
2,ABW,NaN,NaN,90853.0,2000
3,ABW,NaN,NaN,101669.0,2010
4,AFG,0.55,0.09,NaN,1960
...,...,...,...,...,...
899,ZMB,7.35,7.05,13605984.0,2010
900,ZWE,3.29,2.48,NaN,1960
901,ZWE,6.34,5.11,7408624.0,1980
902,ZWE,9.03,8.85,11881477.0,2000


# WIP: ignore

In [5]:
import wbdata

In [28]:
df = pd.DataFrame()
for year in [1960, 1980, 2000, 2010]:
    display(wbdata.get_dataframe({"BAR.SCHL.2024": "edu_avg_20_24", "BAR.SCHL.2024.FE": "fe_edu_avg_20_24"}, date=str(year)).head())

,edu_avg_20_24,fe_edu_avg_20_24
country,,
Anguilla,NaN,NaN
Cook Islands,NaN,NaN
Global Partnership for Education,NaN,NaN
Lending category not classified,NaN,NaN
Mayotte,NaN,NaN


,edu_avg_20_24,fe_edu_avg_20_24
country,,
Anguilla,NaN,NaN
Cook Islands,NaN,NaN
Global Partnership for Education,NaN,NaN
Lending category not classified,NaN,NaN
Mayotte,NaN,NaN


,edu_avg_20_24,fe_edu_avg_20_24
country,,
Anguilla,NaN,NaN
Cook Islands,NaN,NaN
Global Partnership for Education,NaN,NaN
Lending category not classified,NaN,NaN
Mayotte,NaN,NaN


,edu_avg_20_24,fe_edu_avg_20_24
country,,
Anguilla,NaN,NaN
Cook Islands,NaN,NaN
Global Partnership for Education,NaN,NaN
Lending category not classified,NaN,NaN
Mayotte,NaN,NaN
